In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.4 MB/s e

In [ ]:
!pip install tensorflow keras google-auth requests gradio


In [ ]:
import pprint
import google.generativeai as palm

In [ ]:
palm.configure(api_key='your api key')

In [ ]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [ ]:
!pip install google-auth requests



In [ ]:
import os
from google.colab import files

# Upload the JSON key file
uploaded = files.upload()

# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = list(uploaded.keys())[0]


Saving gen-lang-client-0092744598-087bc4a797a8.json to gen-lang-client-0092744598-087bc4a797a8.json


In [ ]:
import json
from google.oauth2 import service_account
import google.auth.transport.requests
import requests

# Path to the service account JSON key file
SERVICE_ACCOUNT_FILE = 'gen-lang-client-0092744598-087bc4a797a8.json'

# Load the service account key file with the appropriate scopes
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/generative-language"]
)

# Refresh the credentials to get an access token
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)
access_token = credentials.token


In [ ]:
def generate_text(prompt):
    url = "https://generativelanguage.googleapis.com/v1beta2/models/text-bison-001:generateText"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": {
            "text": prompt
        },
        "temperature": 0.7,
        "maxOutputTokens": 150,
        "topP": 0.95,
        "topK": 50
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['candidates'][0]['output']
    else:
        return f"Error: {response.status_code}, {response.text}"

# Test the function
prompt = "This is a test prompt for the Google Generative Language API."
generated_text = generate_text(prompt)
print("Generated Text:", generated_text)


Generated Text: Test prompt for the Google Generative Language API.


In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10
!unzip -q animals10.zip
import os

dataset_directory = os.getcwd()
!pwd


Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
 98% 574M/586M [00:07<00:00, 85.8MB/s]
100% 586M/586M [00:07<00:00, 79.9MB/s]
/content


In [ ]:
import os

dataset_directory = '/content'

os.chdir(dataset_directory)


In [ ]:
import os
import numpy as np
import requests
import google.auth.transport.requests
from google.oauth2 import service_account
import gradio as gr
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import shutil

# Path to the service account JSON key file
SERVICE_ACCOUNT_FILE = 'gen-lang-client-0092744598-087bc4a797a8.json'

# Load the service account key file with the appropriate scopes
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/generative-language"]
)

def get_access_token(credentials):
    # Refresh the credentials to get an access token
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    return credentials.token

# Data preparation function
def prepare_data(base_dir, train_dir, val_dir, test_size=0.2, limit_per_category=500):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    categories = os.listdir(base_dir)
    print(f"Categories found: {categories}")
    for category in categories:
        category_path = os.path.join(base_dir, category)
        if not os.path.isdir(category_path):
            continue

        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        print(f"Found {len(images)} images in category '{category}'")
        if len(images) == 0:
            print(f"No images found in category '{category}'")
            continue

        # Limit the number of images per category for initial training
        if len(images) > limit_per_category:
            images = images[:limit_per_category]

        train_images, val_images = train_test_split(images, test_size=test_size)

        train_category_dir = os.path.join(train_dir, category)
        val_category_dir = os.path.join(val_dir, category)

        if not os.path.exists(train_category_dir):
            os.makedirs(train_category_dir)
        if not os.path.exists(val_category_dir):
            os.makedirs(val_category_dir)

        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_category_dir, img))
        for img in val_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(val_category_dir, img))

base_dir = 'raw-img'
train_dir = 'animals-10/train'
val_dir = 'animals-10/validation'

prepare_data(base_dir, train_dir, val_dir)

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Reduced image size for faster processing
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),  # Reduced image size for faster processing
    batch_size=32,
    class_mode='categorical'
)

# Fine-tune MobileNetV2 model
def build_mobilenetv2_model(input_shape, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (128, 128, 3)  # Reduced image size for faster processing
num_classes = len(train_generator.class_indices)  # Automatically determine number of classes
model = build_mobilenetv2_model(input_shape, num_classes)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,  # Reduced number of epochs for initial training
    callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True),
        ModelCheckpoint("animals10_mobilenetv2.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
    ]
)

# Load best weights
model.load_weights("animals10_mobilenetv2.h5")

# Preprocess the images
def preprocess_image(img):
    img = img.resize((128, 128))  # Reduced image size for faster processing
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32') / 255.0
    return img

# Predict function
def predict_image(model, img):
    preds = model.predict(img)
    class_idx = np.argmax(preds, axis=1)[0]
    return list(train_generator.class_indices.keys())[class_idx]

# Call Google PaLM API for additional information
def call_google_palm_api(prediction):
    url = "https://generativelanguage.googleapis.com/v1beta2/models/text-bison-001:generateText"
    headers = {
        "Authorization": f"Bearer {get_access_token(credentials)}",
        "Content-Type": "application/json"
    }
    prompt = (
        f"Based on the following image classification result: {prediction}, provide additional context or information suitable for teaching children about this object."
    )
    payload = {
        "prompt": {
            "text": prompt
        },
        "temperature": 0.7,
        "maxOutputTokens": 150,
        "topP": 0.95,
        "topK": 50
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['candidates'][0]['output']
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to classify image and generate text using Google PaLM API
def classify_and_generate_text(image):
    img = preprocess_image(image)
    prediction = predict_image(model, img)
    additional_info = call_google_palm_api(prediction)
    return prediction, additional_info

# Gradio interface
iface = gr.Interface(
    fn=classify_and_generate_text,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Additional Info")
    ],
    title="Animal Explorer for Kids",
    description="An interactive tool to help kids learn about different animals through image classification using a fine-tuned MobileNetV2 model, with additional descriptions generated using the Google PaLM API."
)

# Launch the Gradio interface
iface.launch()


Categories found: ['sheep', 'elephant', 'chicken', 'spider', 'dog', 'butterfly', 'squirrel', 'cow', '.ipynb_checkpoints', 'horse', 'cat']
Found 1820 images in category 'sheep'
Found 1446 images in category 'elephant'
Found 3098 images in category 'chicken'
Found 4821 images in category 'spider'
Found 4863 images in category 'dog'
Found 2112 images in category 'butterfly'
Found 1862 images in category 'squirrel'
Found 1866 images in category 'cow'
Found 0 images in category '.ipynb_checkpoints'
No images found in category '.ipynb_checkpoints'
Found 2623 images in category 'horse'
Found 1668 images in category 'cat'
Found 4954 images belonging to 11 classes.
Found 2455 images belonging to 11 classes.
Epoch 1/10
154/154 [==============================] - ETA: 0s - loss: 1.0153 - accuracy: 0.6957
Epoch 1: val_accuracy improved from -inf to 0.88734, saving model to animals10_mobilenetv2.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


154/154 [==============================] - 135s 844ms/step - loss: 1.0153 - accuracy: 0.6957 - val_loss: 0.3543 - val_accuracy: 0.8873
Epoch 2/10
154/154 [==============================] - ETA: 0s - loss: 0.6522 - accuracy: 0.7924
Epoch 2: val_accuracy improved from 0.88734 to 0.90296, saving model to animals10_mobilenetv2.h5
154/154 [==============================] - 118s 768ms/step - loss: 0.6522 - accuracy: 0.7924 - val_loss: 0.2914 - val_accuracy: 0.9030
Epoch 3/10
154/154 [==============================] - ETA: 0s - loss: 0.6033 - accuracy: 0.8074
Epoch 3: val_accuracy improved from 0.90296 to 0.91694, saving model to animals10_mobilenetv2.h5
154/154 [==============================] - 128s 830ms/step - loss: 0.6033 - accuracy: 0.8074 - val_loss: 0.2476 - val_accuracy: 0.9169
Epoch 4/10
154/154 [==============================] - ETA: 0s - loss: 0.5457 - accuracy: 0.8192
Epoch 4: val_accuracy improved from 0.91694 to 0.92640, saving model to animals10_mobilenetv2.h5
154/154 [=======